In [9]:
import pandas as pd
import re

In [84]:
tramontano_data_file = '../data/tramontano2018/41564_2018_123_MOESM3_ESM.xlsx'
strain_table = pd.read_excel(tramontano_data_file, 
                             sheet_name = 'S1. Selected gut bacteria')
media_table = pd.read_excel(tramontano_data_file, 
                             sheet_name = 'S2. Media composition',
                             skiprows = 2)
media_conditions = media_table.columns
media_conditions = [cond for cond in media_conditions if cond not in ['Class','Compound']]
media_table = media_table.fillna(0) # replace NA with 0
annotated_growth_data = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S3. Annotated data',
                                      skiprows = 2)
# Load the growth matrix. When growth was observed (based on thresholding),
# mean and (stdev) are included. When growth was not observed, 0 was entered
growth_matrix = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S4. Growth matrix',
                                      index_col=0)
# Load sample size sheet. Really unclear what the format is here. Looks like
# # of biological replicates in first and second batch, but not sure
# what the number in parentheses means.
sample_sizes = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S5. Sample sizes',
                                      index_col=0)

# additional sheets from the data aren't needed

In [86]:
growth_matrix.index

Index(['A. muciniphila', 'A. odontolyticus', 'A. putredinis', 'A. shahii',
       'B. adolescentis', 'B. animalis subsp. lactis BI-07',
       'B. animalis subsp. lactis BL-04', 'B. caccae', 'B. clarus',
       'B. coprocola', 'B. crossotus', 'B. dorei', 'B. eggerthii',
       'B. fragilis', 'B. fragilis enterotoxigenic', 'B. fragilis HM-20',
       'B. fragilis HM-709', 'B. fragilis HM-710', 'B. fragilis HM-711',
       'B. fragilis HM-712', 'B. fragilis HM-713', 'B. fragilis HM-714',
       'B. hansenii', 'B. longum subsp. infantis', 'B. longum subsp. longum',
       'B. obeum', 'B. ovatus', 'B. stercoris', 'B. thetaiotaomicron',
       'B. uniformis', 'B. uniformis HM-715', 'B. uniformis HM-716',
       'B. vulgatus', 'B. vulgatus HM-720', 'B. wadsworthia',
       'B. xylanisolvens', 'C. aerofaciens', 'C. bolteae', 'C. catus',
       'C. comes', 'C. leptum', 'C. perfringens C36', 'C. perfringens S107',
       'C. ramosum', 'C. saccharolyticum', 'D. formicigenerans', 'D. piger',
    

In [144]:
study_organisms = ['B. longum subsp. infantis',
                  'B. longum subsp. longum',
                  'R. gnavus',
                  'C. saccharolyticum',
                  'L. plantarum',
                  'L. lactis',
                  'B. vulgatus HM-720',
                  'B. clarus',
                  'P. merdae',
                  'P. distasonis']

In [145]:
# save the strain data for only the strains we've selected
study_strain_table = strain_table.loc[strain_table['designation in screen'].isin(study_organisms)]
study_strain_table.to_csv('../data/study_strain_subset.tsv',sep='\t')

In [147]:
# WHY DOESN'T B INFANTIS SHOW UP?!?!?!?!?
study_strain_table

,designation in screen,strain,TaxID,source,metabolic model,source of metabolic model,assembly,selection criteria,pre-inoculation media
8,B. clarus,"Bacteroides clarus A 20, YIT 12056, DSM 22519,...",762984,DSM 22519,Bacteroides_clarus_YIT_12056,"Magnusdottir et al., 2016",GCA_000195615.1_ASM19561v1,>= 10^-2 rel. abundance (in at least one samp...,mGAM
24,B. longum subsp. longum,Bifidobacterium longum subsp. longum E194b (Va...,1679,DSM 20219,no,no,NT5028,>= 10^-2 rel. abundance (in at least one samp...,BHI++
33,B. vulgatus HM-720,"Bacteroides vulgatus CL09T03C04, HM-720",997891,HM-720 (BEI Resources),no,no,GCA_000273295.1_Bact_vulg_CL09T03C04_V1,>= 10^-2 rel. abundance (in at least one samp...,mGAM
44,C. saccharolyticum,"Clostridium saccharolyticum WM1, DSM 2544, ATC...",610130,DSM 2544,no,no,GCA_000144625.1_ASM14462v1,forming separate metabolic clade represented b...,GMM
67,L. lactis,"Lactococcus lactis subsp. lactis IL1403, CIRM ...",272623,CIRM BIA 96 = CNRZ 1342 (INRA collection),Lactococcus lactis subsp. lactis IL1403,http://systemsbiology.ucsd.edu/InSilicoOrganis...,GCA_000006865.1_ASM686v1,probiotic,mGAM
69,L. plantarum,"Lactobacillus plantarum WCFS1, LMG 9211, NCIMB...",220668,LMG 9211 (BCCM collection),Lactobacillus_plantarum_WCFS1,"Magnusdottir et al., 2016",GCA_000203855.3_ASM20385v3,probiotic,mGAM
78,P. distasonis,"Parabacteroides distasonis DSM 20701, ATCC 850...",435591,DSM 20701,Parabacteroides_distasonis_ATCC_8503,"Magnusdottir et al., 2016",GCA_000012845.1_ASM1284v1,>= 10^-2 rel. abundance (in at least one samp...,mGAM
80,P. merdae,"Parabacteroides merdae VPI T4-1, DSM 19495, AT...",411477,DSM 19495,Parabacteroides_merdae_ATCC_43184,"Magnusdottir et al., 2016",GCA_000154105.1_ASM15410v1,>= 10^-2 rel. abundance (in at least one samp...,mGAM
82,R. gnavus,"Ruminococcus gnavus VPI C7-9, ATCC 29149",411470,ATCC 29149,Ruminococcus_gnavus_ATCC_29149,"Magnusdottir et al., 2016",GCA_000169475.1_ASM16947v1,>= 10^-2 rel. abundance (in at least one samp...,GMM


In [62]:
# may want to do some cleaning of the media table at a later time. Difficult to parse w/ g/mg/ug/ml
# units dispersed throughout the dataframe

media_table = media_table.fillna(0) # replace NA with 0
# conditions_only = media_table[media_conditions]

# for condition in media_conditions:
#     condition_series = conditions_only[condition]
#     condition_series.str.find('g')
# #conditions_only.find

# # convert grams to micrograms
# media_table = media_table.replace(to_replace=r'\s[g]', value=r'000000', regex=True)
# # convert mg to micrograms
# media_table = media_table.replace(to_replace=r'\s[mg]', value='000', regex=True)
# # strip microgram label
# media_table = media_table.replace(to_replace=r'\s[µg]', value='', regex=True)
# # convert masses from string to numeric

In [99]:
# which conditions had the most positive growth for these 10 species?
(growth_matrix.loc[study_organisms] != 0).sum().sort_values()

M9        0
M1        3
M16       4
M15A      4
M14       4
M13       4
M15B      4
M10       5
M7        7
M4        7
M3        8
M2        8
M11       9
mGAM      9
M5        9
M8       10
WCA      10
BHI++    10
GMM      10
dtype: int64

In [ ]:
# exclude M5, M11 for now just to limit conditions since these are just metabolite removals.
# 

In [107]:
top = ['WCA','M8','M2','M3']
growth_matrix.loc[study_organisms, top]

,WCA,M8,M2,M3
designation in screen,,,,
B. longum subsp. infantis,0.588 (0.07),1.478 (0.041),1.562 (0.04),1.256 (0.019)
B. longum subsp. longum,0.429 (0.142),0.288 (0.073),0.707 (0.52),0
R. gnavus,0.146 (0.035),0.733 (0.063),0.66 (0.245),0.764 (0.014)
C. saccharolyticum,0.526 (0.305),0.862 (0.056),1.269 (0.313),0.576 (0.147)
L. plantarum,0.675 (0.329),0.765 (0.13),1.092 (0.548),0.858 (0.06)
L. lactis,0.802 (0.562),0.534 (0.045),0.746 (0.434),0.835 (0.425)
B. vulgatus HM-720,0.31 (0.02),0.999 (0.106),0.615 (0.044),0.82 (0.157)
B. clarus,0.694 (0.319),0.756 (0.104),0.655 (0.372),0
P. merdae,0.21 (0.003),0.491 (0.051),0,0.204 (0.123)


In [108]:
# These conditions will get at least 3 positive growth conditions per species, and all should grow in
# a 50/50 mix of spent WCA+ fresh WCA from R. gnavus and P. distasonis.

In [126]:
# extract the media table for only the media selected, then remove
# ingredients that aren't present for any of these media.
# WCA is ordered as a dehydrated standalone mixture, so just get the defined media.
top_in_media_table_nomenclature = ['Class','Compound','M8: GMM+LAB plus Mucin','M2: LAB','M3: GMM+LAB']
subset = media_table[top_in_media_table_nomenclature]
subset = subset.loc[(subset == 0).sum(axis=1) < 3]
subset.to_csv('../data/media_subsetted.tsv', sep='\t')
subset

,Class,Compound,M8: GMM+LAB plus Mucin,M2: LAB,M3: GMM+LAB
0,Sugar,D-glucose,5 g,15 g,5 g
1,Sugar,Fructose,1 g,0,1 g
2,Sugar,Cellobiose,1 g,0,1 g
3,Sugar,Maltose,1 g,0,1 g
4,Sugar,Lactose,1 g,0,1 g
6,Others,Resazurin,1 mg,0,1 mg
7,Others,NAD (N1511),2 mg,0,2 mg
8,Others,Hemin,0.5 mg,0,0.5 mg
9,Others,Mucin,5 g,0,0
10,Others,KCl,50 mg,0,0


In [130]:
# Load the subsetted table that we manually added catalog info for:
subset_w_catalog = pd.read_csv('../data/media_subsetted_with_catalog.txt', sep='\t')
# get the subset of items we're going to buy
to_buy = subset_w_catalog.loc[subset_w_catalog['Buy?'] == 'Yes']
to_buy.to_csv('../data/media_subsetted_for_purchase.tsv', sep='\t')
to_buy

,Unnamed: 0,Class,Compound,M8: GMM+LAB plus Mucin,M2: LAB,M3: GMM+LAB,price,volume/mass (ml/g),price per unit,Catalog ID,link,Notes,Guerrant stock,Buy?
3,3.0,Sugar,Maltose,1 g,0,1 g,$35.28,100.0,$0.35,VWRV1B1184-100G,https://us.vwr.com/store/product/7437397/d-mal...,NaN,NaN,Yes
6,7.0,Others,NAD (N1511),2 mg,0,2 mg,$155,1.0,$155.00,N1511-1G,https://www.sigmaaldrich.com/catalog/product/s...,Smaller quantities available,NaN,Yes
7,8.0,Others,Hemin,0.5 mg,0,0.5 mg,$53.60,1.0,$53.60,51280-1G,https://www.sigmaaldrich.com/catalog/product/s...,NaN,NaN,Yes
8,9.0,Others,Mucin,5 g,0,0,$368.00,100.0,$3.68,M1778-10G,https://www.sigmaaldrich.com/catalog/product/s...,Smaller quantities available,NaN,Yes
10,12.0,Others,Hematin,1.2 mg,0,1.2 mg,$48.90,1.0,$48.90,H3281-1G,https://www.sigmaaldrich.com/catalog/product/s...,Larger quantities available,NaN,Yes
11,13.0,Amino acids,l-Histidine,0.171 g,0.17 g,0.171 g,$27.60,25.0,$1.10,H8000-25G,https://www.sigmaaldrich.com/catalog/product/s...,Larger quantities available,NaN,Yes
12,14.0,Amino acids,l-Isoleucine,0.24 g,0.24 g,0.24 g,$50.50,25.0,$2.02,I2752-25G,https://www.sigmaaldrich.com/catalog/product/s...,NaN,NaN,Yes
14,16.0,Amino acids,l-Methionine,0.125 g,0.125 g,0.125 g,$22.50,25.0,$0.90,M9625-25G,https://www.sigmaaldrich.com/catalog/product/s...,NaN,NaN,Yes
15,17.0,Amino acids,l-Valine,0.7 g,0.7 g,0.7 g,$25.60,25.0,$1.02,V0500-25G,https://www.sigmaaldrich.com/catalog/product/s...,NaN,NaN,Yes
17,19.0,Amino acids,l-Cysteine,0.5 g,0.2 g,0.5 g,$52.60,25.0,$2.10,C7352-25G,https://www.sigmaaldrich.com/catalog/product/s...,NaN,"100g, could use some more",Yes


In [91]:
# GMM, BHI++, WCA, and mGAM are the rich media used (the rest are defined).
# all other media are the only ones present in the media_table.

{'Amino acids',
 'Buffer compounds',
 'Nucleotids',
 'Others',
 'SCFA',
 'Salts & Minerals',
 'Sugar',
 'Vitamins & Antioxidants'}

In [92]:
media_table.columns

Index(['Class', 'Compound', 'M1: GMM', 'M2: LAB', 'M3: GMM+LAB',
       'M4: GMM+LAB low M/V', 'M5: GMM+LAB exclude SCFA',
       'M7: GMM+LAB only monosacharides', 'M8: GMM+LAB plus Mucin',
       'M9: GMM+LAB only Mucin', 'M10: GMM+LAB 10% aminoacids',
       'M11: GMM+LAB excluding aromatic AA', 'M13: B. thetaiotaomicron MM',
       'M14: C. perfiringens MM', 'M15A: E. coli MM1', 'M15B: E. coli MM2',
       'M16: V. parvula defined medium'],
      dtype='object')

In [94]:
growth_matrix.columns

Index(['GMM', 'BHI++', 'WCA', 'mGAM', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8',
       'M9', 'M10', 'M11', 'M13', 'M14', 'M15A', 'M15B', 'M16'],
      dtype='object')